In [87]:
import ConfigParser
import email
import imaplib
import logging
import logging.config
import os
import pandas as pd
import re
import yaml

from datetime import datetime
from lxml import etree

imaplib.IMAP4.debug = imaplib.IMAP4_SSL.debug = 1

In [88]:
def configure_logging(default_path='logging.yml', default_level=logging.INFO):
    path = default_path

    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = yaml.safe_load(f.read())
            logging.config.dictConfig(config)
    else:
        logging.basicConfig(level=default_level)

In [89]:
# the current logging configuration is logging to the console, change to log to a file with rotation before deploying
configure_logging()
logger = logging.getLogger()
logger.debug('TEST')

2018-07-20 14:11:20,379 - root - DEBUG - TEST


In [90]:
config = ConfigParser.SafeConfigParser()
config.read('config.ini')

['config.ini']

In [91]:
def prepare(config):
    if not os.path.exists(config.get('DEFAULT', 'sandbox')):
        os.makedirs(config.get('DEFAULT', 'sandbox'))
        
    if not os.path.exists(config.get('DEFAULT', 'output')):        
        os.makedirs(config.get('DEFAULT', 'output'))

In [92]:
prepare(config)

# Workflow
## Download Attachments
1. Excel file arrives as email attachment to a specific address @archives-africa.org (tbd).
2. Checks to make sure it has arrived from a list of known addresses (needs to be configurable using a command-line text editor).
1. Excel file saved somewhere on the Archives Africa server.
1. File renamed using a sequential numbering system (to be discussed) and a code representing the sender (again needs to be configurable). This new filename also used for XML data files (below).

In [93]:
def is_email_address_known(config, address):
    return config.has_section(address)

In [94]:
print(is_email_address_known(config, 'test@test.me'))
print(is_email_address_known(config, 'email@archive1.org'))

False
True


In [95]:
def save_attachment(config, address, attachment):
    sandbox = config.get('DEFAULT', 'sandbox')
    
    code = config.get(address, 'code')

    seq = 1
    if config.has_option(address, 'sequence'):
        seq = config.getint(address, 'sequence') + 1
    
    filename = os.path.join(sandbox, '{}_{}.xlsx'.format(code, seq))
    
    with open(filename, 'w') as f:
        f.write(attachment)
        f.close()

    return seq, filename

In [96]:
def update_sequence(config, filename, address, seq):
    config.set(address, 'sequence', str(seq))
    
    with open(filename, 'wb') as f:
        config.write(f)

In [97]:
address = 'email@archive1.org'

with open('draft_01.xlsx', 'r') as f:
    attachment = f.read()

seq, filename = save_attachment(config, address, attachment)

print(seq, filename)

(6, 'sandbox/dir/A1_6.xlsx')


In [98]:
update_sequence(config, 'config.ini', address, seq)

In [99]:
# this code has not been tested, no mailbox available at the time of writing
def get_data_from_mailbox(config):
    logger = logging.getLogger()
    
    username = config.get('mailbox', 'username')
    password = config.get('mailbox', 'password')

    
    logger.debug('Connecting via ssl IMAP...')
    mailbox = imaplib.IMAP4_SSL('host', 993)
    logger.debug('Conected!!')
    
    logger.debug('Trying to login via ssl')
    mailbox.login(username, password)
    logger.debug('logged in!!')
    
    logger.debug('Selecting INBOX')
    mailbox.select()
    logger.debug('Getting unread messages...')
    typ, data = con.search(None, '(UNSEEN)')

    counter = 0
    
    for num in data[0].split():
        typ, data = con.fetch(num, '(RFC822)')

        text = data[0][1]
        msg = email.message_from_string(text)
        
        sender = msg['From']
        
        # skips unknown addresses
        if not is_email_address_known(config, sender):
            continue
        
        for part in msg.walk():
            if part.get_content_maintype() == 'multipart':
                continue

            if part.get('Content-Disposition') is None:
                continue

            data = part.get_payload(decode=True)
            if not data:
                logger.debug('{}: No attachments found...'.format(sender))
                continue
            
            seq, filename = save_attachment(config, sender, data)
            logger.info('{}: Downloaded attachment {}'.format(sender, filename))
            
            update_sequence(config, 'config.ini', address, seq)
            
            counter += 1
        
        logger.info('Downloaded {} attachement(s)'.format(counter))

    con.close()
    con.logout()

## Process Downloaded Attachments
5. Language of Excel file to be determined (either English or French) – can embed a hidden field in the Excel file if helpful. ALL MESSAGES TO THE SENDER OF FILE NEED TO BE IN THIS LANGUAGE. Ideally messages should be configurable (using a command-line text editor).
5. Character set converted to UTF-8 if not already. All files past this point should be UTF-8.
5. Make sure that all required* fields are present.

5. If all required COLLECTION fields are present and correct, then:
    1. Content extracted from the Excel file into separate XML files – one XML file for each worksheet. Exact XML schemas to be defined – but as simple as possible. One schema for COLLECTION data. Another simple schema for TERMS.
    1. COLLECTION data cleaned: All data wrapped with <p> tags, including line-breaks between paragraphs in multi-line fields.
    1. Dates converted to ISO-8601 format (YYYY-MM-DD, YYYY-MM or YYYY) where possible in several specific fields (tbd).
    1. We may need to see if it’s possible to extract styling information like bold and italic text – and convert this to HTML tags `<strong>` and `<em>`. If any `<strong>` or `<em>` tags present, wrap the entire field data with `<![CDATA[…]]>`
    1. COLLECTION data saved as simple XML file (schema to be discussed).
    1. TERMS data (worksheets 2 – 5): Also parsed, cleaned and saved to simple XML files.

5. If all files saved successfully (no errors), then:
    1. ‘File processed’ report sent (in EN) to Archives Africa with filenames/locations, timestamps, originating email address etc.
    1. Also, simple ‘thank you for sending your data to Archives Africa – we will let you know when it has been added to the site’ (tbd) message (in EN or FR) to sender of file.
    1. If any failures – missing fields, failures during cleaning, then email message sent back to originating address (in EN or FR) with details of problems found (missing fields etc). This mail also sent (separately) to an address @archives-africa.org (tbd).


In [101]:
def clean_collection(filename):
    collection = pd.read_excel(filename, 'collection')
    collection.drop('ARCHIVES AFRICA: COLLECTION DATA', axis=1).drop(0)
    
    collection = collection.transpose()
    new_header = collection.iloc[0].str.strip()
    collection = collection[1:]
    collection.columns = new_header
    
    collection.drop(['* Required'], axis=1, inplace=True)

    return collection

In [102]:
collection = clean_collection(filename)
collection.head()

ARCHIVES AFRICA: COLLECTION DATA,Institution identifier*,Repository*,Collection identifier*,Title*,Collection creation date*,Record creation date,Record revision date,Record deletion date,Level of description*,Extent and medium*,...,Scope and content*,Accruals,System of arrangement*,Conditions governing access*,Conditions governing reproduction,Language of material*,Finding aids,Related units of description,Notes (PUBLIC): Please note: These notes WILL be publicaly viewable.,Archivist's notes (PRIVATE): Please include name of archivist. These notes will NOT be publicaly viewable.
Unnamed: 1,<institution_id>,<repository_name>,<collection_id>,<title>,<date_creation_collection>,<date_creation_record>,<date_revision_record>,<date_deletion_record>,<level_of_description>,<extent_medium>,...,<scope_content>,<accruals>,<arrangement>,<conditions_access>,<conditions_reproduction>,<language_material>,<finding_aids>,<related_descriptions>,<notes_public>,<notes_private>
Unnamed: 2,Example identifier,Example repository,Example collection ID,Example title,Example collection creation date,2018-03-19 00:00:00,2018-03-20 00:00:00,2018-03-21 00:00:00,Example level of description,Example extent,...,Example scope and content. Multi-line data - m...,Example accruals. Multi-line data - multi-para...,Example system of arrangement,Example conditions governing access,Example conditions governing reproduction,Example language of materials,Example finding aids,Example related units of description,Example public notes. Multi-line data - multi-...,Example private notes. Multi-line data - multi...
Unnamed: 3,</institution_id>,</repository_name>,</collection_id>,</title>,</date_creation_collection>,</date_creation_record>,</date_revision_record>,</date_deletion_record>,</level_of_description>,</extent_medium>,...,</scope_content>,</accruals>,</arrangement>,</conditions_access>,</conditions_reproduction>,</language_material>,</finding_aids>,</related_descriptions>,</notes_public>,</notes_private>
Examples:,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,...,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied,To be supplied
For more information:,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.ica.org/en/isdiah-international-st...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,NaN,NaN,NaN,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,https://www.accesstomemory.org/en/docs/2.4/use...,NaN


In [103]:
collection['Institution identifier*'][1], collection['Repository*'][1]

(u'Example identifier', u'Example repository')

In [104]:
def get_missing_fields(collection):
    missing_fields = []

    for c in collection.columns:
        # required fields
        if '*' in c:
            data = collection[c][1]
            if pd.isna(data) or pd.isnull(data):
                missing_fields.append(c)
    
    return missing_fields

In [105]:
missing_fields = get_missing_fields(collection)
if missing_fields:
    print(missing_fields)

In [106]:
def collection_to_xml(collection):
    xml = etree.Element('collection')
    
    for c in collection.columns:
        data = collection[c][1]
        
        if not pd.isna(data) and not pd.isnull(data):
            name = collection[c][0]
            name = re.sub(r'\W', '', name)
            el = etree.SubElement(xml, name)
            
            if isinstance(data, str):
                paras = data.split('\n')
                
                for para in paras:
                    para = para.strip()
                    if para:
                        p = etree.SubElement(el, 'p')
                        if '<' in para or '>' in para:
                            p.text = etree.CDATA(para)
                        else:
                            p.text = para
            elif isinstance(data, datetime):
                data = data.date().isoformat()
                el.text = data
            else:
                el.text = data
    
    return etree.ElementTree(xml)

In [107]:
xml = collection_to_xml(collection)
# etree.dump(xml.getroot())

In [135]:
def save_xml(xml, name):
    logger.debug('save_xml: {}'.format(name))
    xml.write(name, encoding='utf-8', method='xml', pretty_print=True)

In [136]:
save_xml(xml, 'collection.xml')

2018-07-20 14:20:14,695 - root - DEBUG - save_xml: collection.xml


In [137]:
def process_collection(collection, filename):
    xml = collection_to_xml(collection)
    save_xml(xml, filename)  

In [138]:
def terms_to_xml(terms, root):
    xml = etree.Element(root)
    
    for term in terms:
        if not pd.isna(term) and not pd.isnull(term):
            p = etree.SubElement(xml, 'p')
            p.text = term
            
    return etree.ElementTree(xml)

In [145]:
def process_terms(xlsx, path):
    excel = pd.ExcelFile(xlsx)
    for sn in excel.sheet_names[1:]:
        df = pd.read_excel(data, sn)
        term_name = df.columns[0]
        el_name = term_name.lower().replace(' ', '_')

        xml = terms_to_xml(df[term_name], el_name)
        
        filename = os.path.basename(xlsx).split('.')[0]
        save_xml(
            xml, os.path.join(path, '{}_{}.xml'.format(filename, el_name)))

In [148]:
def process_attachments(config):
    logger = logging.getLogger()
    in_path = config.get('DEFAULT', 'sandbox')
    out_path = config.get('DEFAULT', 'output')
    
    for root, dirs, files in os.walk(in_path):
        for name in files:
            filepath = os.path.join(in_path, name)
            collection = clean_collection(filepath)

            missing_fields = get_missing_fields(collection)
            if missing_fields:
                logger.warn('{}: has missing fields {}'.format(
                    name, missing_fields))
                # TODO: generate failure report
                continue
            
            filename = os.path.basename(name).split('.')[0]
            process_collection(
                collection, os.path.join(out_path, '{}.xml'.format(filename)))
            process_terms(filepath, out_path)
            
            logger.info('{}: processed'.format(name))
            
            # TODO: generate success reports

In [149]:
process_attachments(config)

2018-07-20 14:24:07,792 - root - DEBUG - save_xml: output/dir/A1_6.xml
2018-07-20 14:24:07,837 - root - DEBUG - save_xml: output/dir/A1_6_subjects.xml
2018-07-20 14:24:07,862 - root - DEBUG - save_xml: output/dir/A1_6_places.xml
2018-07-20 14:24:07,880 - root - DEBUG - save_xml: output/dir/A1_6_personal_names.xml
2018-07-20 14:24:07,903 - root - DEBUG - save_xml: output/dir/A1_6_corporate_names.xml
2018-07-20 14:24:07,905 - root - INFO - A1_6.xlsx: processed
2018-07-20 14:24:07,967 - root - DEBUG - save_xml: output/dir/A1_3.xml
2018-07-20 14:24:08,002 - root - DEBUG - save_xml: output/dir/A1_3_subjects.xml
2018-07-20 14:24:08,022 - root - DEBUG - save_xml: output/dir/A1_3_places.xml
2018-07-20 14:24:08,041 - root - DEBUG - save_xml: output/dir/A1_3_personal_names.xml
2018-07-20 14:24:08,066 - root - DEBUG - save_xml: output/dir/A1_3_corporate_names.xml
2018-07-20 14:24:08,069 - root - INFO - A1_3.xlsx: processed
2018-07-20 14:24:08,128 - root - DEBUG - save_xml: output/dir/A1_5.xml
201